In [2]:
import csv
from qiskit import transpile
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_distribution, plot_histogram
from qiskit_aer import AerSimulator
import qiskit.quantum_info as qi
from qiskit.providers.fake_provider import GenericBackendV2
import numpy as np
from quantum_check import construct_circuit, check_consistency
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
#Load in data from data.csv
test_data = []

with open("data/data.csv", mode="r", newline="") as file: 
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        n = int(row[0])
        system_type = row[1]
        F1, F2, F3 = row[2], row[3], row[4]
        test_data.append((n, system_type, F1, F2, F3))

In [37]:
def experimental_quantum_test():
    exp = 0
    terms = []
    func = 0
    var = 0
    label = ''

    for data in test_data:
        exp += 1
        var = data[0]
        label = data[1]
        for i in range(len(data) - 2):
            terms.append(data[i+2])
        func = len(terms)
        
        circ = construct_circuit(terms, var, func)

        backend = GenericBackendV2(len(circ.qubits))
        circ = transpile(circ, backend)

        shots = [64, 128, 256]
        
        a_probs = []
        a_probs.append(exp)
        a_probs.append(var)
        a_probs.append(label)

        for shot in shots:
            result = backend.run(circ, shots=shot)
            probabilities = {k:v/shot for k,v in result.result().get_counts().items()}
            
            a_probs.append(probabilities.get('0000', 0))
            a_probs.append(probabilities.get('0011', 0))
            a_probs.append(probabilities.get('1100', 0))
            a_probs.append(probabilities.get('1111', 0))
        
        with open('data/experimental_probabilities.csv', mode='a', newline='') as file: #64x000,64x0011,64x1100,64x1111,128x000,128x0011,128x1100,256x1111,256x000,256x0011,256x1100,256x1111
            writer = csv.writer(file)
            writer.writerow(a_probs)
        
        terms = []

In [ ]:
for i in range(5):
    experimental_quantum_test()

In [44]:
#Data analysis
df = pd.read_csv("data/experimental_probabilities.csv")

df_sorted = df.sort_values(by='experiment')

columns_to_exclude = ['var', 'type']
numeric_df = df_sorted.drop(columns=columns_to_exclude)
df_avg_numeric = numeric_df.groupby('experiment').mean().reset_index()
meta_cols = df_sorted.groupby('experiment')[columns_to_exclude].first().reset_index()

df_avg = pd.merge(df_avg_numeric, meta_cols, on='experiment')
df_avg.to_csv('data/experimental_mean_results.csv', index=False)
df_avg = pd.merge(df_avg_numeric, meta_cols, on='experiment')
ordered_cols = ['experiment', 'var', 'type'] + [col for col in df_avg.columns if col not in ['experiment', 'var', 'type']]

df_avg = df_avg[ordered_cols]
df_avg.to_csv('data/experimental_mean_results.csv', index=False)

std_df = df.groupby(['experiment', 'var', 'type']).std().reset_index()
std_df.to_csv('data/experimental_std_results.csv', index=False)


In [ ]:
#Graph creation
df_theo = pd.read_csv('data/theoretical_mean_results.csv')
df_exp_mean = pd.read_csv('data/experimental_mean_results.csv')
df_exp_std = pd.read_csv('data/experimental_std_results.csv')

labels = ['0000', '0011', '1100', '1111']
x = np.arange(len(labels))

output_rows = []
output_dir = "figures/experimental_figures"

for i, (row_theo, row_exp_mean, row_exp_std) in enumerate(zip(df_theo.iterrows(), df_exp_mean.iterrows(), df_exp_std.iterrows())):
    _, row_theo = row_theo
    _, row_exp_mean = row_exp_mean
    _, row_exp_std = row_exp_std

    var_value = row_theo['var']
    type_value = row_theo['type']

    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    for j in range(3):
        start_idx = 3 + j * 4
        theo_vals = row_theo[start_idx:start_idx + 4].values.astype(float)
        exp_vals = row_exp_mean[start_idx:start_idx + 4].values.astype(float)
        std_vals = row_exp_std[start_idx:start_idx + 4].values.astype(float)

        axs[j].bar(x, theo_vals, width=0.6, label='Theoretical', color='skyblue', alpha=0.7)
        axs[j].bar(x, exp_vals, width=0.3, label='Experimental', color='salmon', alpha=0.9,
                   yerr=std_vals, capsize=5, error_kw={'elinewidth': 1, 'ecolor': 'navy'})
    
        axs[j].set_ylim(bottom=0)
        axs[j].set_title(f"Experiment {i+1} {type_value} - {2**(j+6)} Shots")
        axs[j].set_xlabel('State')
        axs[j].set_ylabel('Quasi-Probabilities')
        axs[j].set_xticks(x)
        axs[j].set_xticklabels(labels)
        axs[j].legend()

        for k in range(4):
            label = labels[k]
            exp = exp_vals[k]
            std = std_vals[k]

            output_rows.append({
                "experiment": i + 1,
                "var": var_value,
                "type": type_value,
                "shots": 2**(j+6),
                "label": label,
                "experimental_mean": exp,
                "std_dev": std,
            })

    plt.tight_layout()
    fig.savefig(f"{output_dir}/experiment_{i+1}_var_{var_value}_{type_value}.png")
    plt.close(fig)

output_df = pd.DataFrame(output_rows)
output_df.to_csv("data/experimental_std_dev_all.csv", index=False)